<a href="https://colab.research.google.com/github/ryandhjeon/pytorch-exercise/blob/main/cnn_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying MNIST with CNN & Deep CNN

In [50]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

### Initialize

In [51]:
# Batch size * Channel * Height * Width
inputs = torch.Tensor(1, 1, 28, 28)
print('Tensor size: {}'.format(inputs.shape))

Tensor size: torch.Size([1, 1, 28, 28])


In [52]:
# Input: 1 channel, output: 32 channel, kernel size: 3, padding: 1
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [53]:
# Input: 32 channel, output: 64 channel, kernel size: 3, padding: 1
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [54]:
# Max pulling
maxpool = nn.MaxPool2d(2)
print(maxpool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


### Modeling

In [55]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [56]:
out = maxpool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [57]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [58]:
out = maxpool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [59]:
out.size(0)

1

In [60]:
out.size(1)

64

In [61]:
out.size(2)

7

In [62]:
out.size(3)

7

In [63]:
# Leave the batch & spread out the rest
out = out.view(out.size(0), -1) 
print(out.shape)

torch.Size([1, 3136])


### Classify MNIST with CNN

In [64]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Set the random seed
torch.manual_seed(777)

# If using GPU, set the random seed 
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [65]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [66]:
mnist_train = dsets.MNIST(root='../dataset/', # Download path
                          train=True, # True: Download training data
                          transform=transforms.ToTensor(), # transforms to tensor
                          download=True)

mnist_test = dsets.MNIST(root='../dataset/', # Download path
                         train=False, # False: Download test data
                         transform=transforms.ToTensor(), # transforms to tensor
                         download=True)

In [67]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [68]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 1st floor
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 2nd floor
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # initialize
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten
        out = self.fc(out)
        return out


In [69]:
# CNN Model
model = CNN().to(device)

In [70]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [71]:
total_batch = len(data_loader)
print('Total # of batches: {}'.format(total_batch)) # 600 * 100 = 60,000

Total # of batches: 600


In [72]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # Grab by mini batch. X: Mini batch, Y: label.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.220139161
[Epoch:    2] cost = 0.0609170943
[Epoch:    3] cost = 0.0460223593
[Epoch:    4] cost = 0.0366800986
[Epoch:    5] cost = 0.0299777929
[Epoch:    6] cost = 0.0260486249
[Epoch:    7] cost = 0.0207757875
[Epoch:    8] cost = 0.0184871405
[Epoch:    9] cost = 0.0154143889
[Epoch:   10] cost = 0.0132803731
[Epoch:   11] cost = 0.0119735664
[Epoch:   12] cost = 0.00869387202
[Epoch:   13] cost = 0.00800391752
[Epoch:   14] cost = 0.00654100487
[Epoch:   15] cost = 0.00728077115


In [73]:
# MNIST test data
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9850999712944031


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


### Classify MNIST with Deep CNN

In [74]:
class DeepCNN(torch.nn.Module):

    def __init__(self):
        super(DeepCNN, self).__init__()
        self.keep_prob = 0.5
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [80]:
model2 = DeepCNN().to(device)

In [81]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)

In [82]:
total_batch = len(data_loader)
print('Total # of batches: {}'.format(total_batch)) # 600 * 100 = 60,000

Total # of batches: 600


In [83]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model2(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


[Epoch:    1] cost = 0.196627051
[Epoch:    2] cost = 0.049133379
[Epoch:    3] cost = 0.034763366
[Epoch:    4] cost = 0.0283150449
[Epoch:    5] cost = 0.0223801285
[Epoch:    6] cost = 0.0178907309
[Epoch:    7] cost = 0.0156221138
[Epoch:    8] cost = 0.0137809915
[Epoch:    9] cost = 0.0130404616
[Epoch:   10] cost = 0.0106102927
[Epoch:   11] cost = 0.0102392584
[Epoch:   12] cost = 0.00824494846
[Epoch:   13] cost = 0.00873505697
[Epoch:   14] cost = 0.00691596279
[Epoch:   15] cost = 0.00748062786


In [84]:
# MNIST test data
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model2(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.920199990272522


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


**Lower Accuracy with Deep CNN. Deep does not mean increasing the accuracy.**